---


#  🟢 **Minicurso** - Processamento e Visualização de Imagens de Satélite, Dados de Precipitação e Índices de Vegetação com Python (PyVisSat)

---

> ## **Aula 1:** $\underline{Plotagem\ de\ Imagens \ de\ Satélite}$

---
**OBJETIVO:**

- Nesta aula prática aprenderemos como baixar imagens de satélite, transformar de projeção geoestacionária para retangular, plotar as imagens em forma de mapas e combinar com informações de relâmpagos dos satélites [GOES-16/19](https://space.oscar.wmo.int/satelliteprogrammes/view/goes_3rd_generation).

---

**DADOS DE ENTRADA**:
- Serão utilizados os arquivos de temperatura de brilho do canal 13 (10.3 µm) do sensor [Advanced Baseline Imager(ABI)](https://space.oscar.wmo.int/instruments/view/abi) dos satélites GOES-16/19 em formato NetCDF disponibilizados pela NOAA no repositório da Amazon].

    -  **Formato do dado:** arquivo NetCDF
    -   **Tipo do dado:** matriz de 5424 linhas x 5424 colunas  
    - **Nome do arquivo:** OR_ABI-L2-CMIPF-M6C01_G16_s20250960000205_e20250960009513_c20250960009571.nc
    - **Fonte dos dados:** [GOES-16](https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-CMIPF/) e [GOES-19](https://noaa-goes19.s3.amazonaws.com/index.html#ABI-L2-CMIPF/)


---

**DADOS DE SAÍDA:**
- Serão produzidas figuras no formato JPG e animações de imagens de satélite em formato GIF.

---

**PROCEDIMENTO REALIZADO:**
- Os seguintes procedimentos são realizados nessa aula:

    1.   **1° Passo:** Instalação das Bibiotecas
    2.   **2° Passo:** Download de Arquivos Auxiliares
    3.   **PARTE 1)** - Projeção Satélite em Níveis de Cinza
    4.   **PARTE 2)**: Projeção Satélite em T-Realçada
    5.   **PARTE 3)**: Projeção Retangular em Níveis de Cinza
    6.   **PARTE 4)**: Projeção Retangular em T-Realçada
    7.   **PARTE 5)**: Plotando Várias Imagens do IR e Criando Animação
    8.   **PARTE 6)**: Plotando Painel de Imagens
    9.   **PARTE 7)**: Projeção Satélite da Imagem Realçada + Total de Flashes do GLM
    10.  **PARTE 8)**: Evolução Temporal da Temperatura de Brilho do IR e Flashes GLM


---
**OBSERVAÇÕES IMPORTANTES**:
1. Necessário possuir uma conta do Gmail.
2. Salvar o código no seu drive. Para isto clicar em `Arquivo` e depois em `Salvar uma cópia no drive` e fazer o login numa conta google.


---

**REALIZADO E MINISTRADO POR:**
- **Realizado por:** Diego Souza/INPE e Prof. Dr. Enrique V. Mattos - 19/08/2024
- **Atualizado por:** Prof Dr. Enrique V. Mattos - 15/11/2025
---

# **1° Passo:** Instalação das Bibliotecas

Note que o Google Colab roda uma máquina Linux na Nuvem que já tem o Python instalado.

In [ ]:
# vamos verificar a versão do python instalado no Google Colab
!python --version

O primeiro passo será instalar as bibliotecas necessárias (e suas dependências) para a execução dos códigos. Basicamente, as bibliotecas terão a seguinte finalidade:

*   `Netcdf4:` Ler os dados de arquivos no formato NetCDF
*   `Cartopy:` Adicionar mapas aos plots
*   `Boto3:` Download de dados GOES-16/19 diretamente da nuvem da Amazon Web Services (AWS)
*   `geobr:` Trabalhar com shapefiles de municípios e estados
*   `salem:` Extrair dados de um shapefile
*   `rasterio, pyproj, geopandas e descartes:` Extrair informações de uma região


In [ ]:
# verificando as bibliotecas instaladas no Colab
!pip list

In [ ]:
# verificando se a biblioteca "boto3" esta instalada no Colab
!pip show boto3

In [ ]:
# instalando as bibliotecas
!pip install -q netcdf4 cartopy boto3 geobr salem rasterio pyproj geopandas descartes

In [ ]:
# verificando se a biblioteca "boto3" foi instalada no Colab
!pip show boto3

# **2° Passo:** Download de Arquivos Auxiliares



Neste passo vamos baixar alguns arquivos auxiliares necessários para parte dos códigos que serão demonstrados nesta aula prática. Os arquivos estão disponíveis no [GitHub](https://github.com/evmpython/Minicurso_UFCG_nov_2025) do nosso curso:

*   `utilities_goes16e19.py:` Código python que contêm algumas funções para processamento de dados do satélite GOES-16/19.
*   `ir.cpt:` Paleta de cores para o canal do infravermelhos do satélite GOES-16/19.

In [ ]:
# download do arquivo "utilities_goes16e19.py"
!wget -c https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/utilities_goes16e19.py

# download da paleta de cores para o canal do infravermelho
!wget -c https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/ir.cpt

#**PARTE 1)**: Projeção Satélite em Níveis de Cinza





---


- Nesta primeira parte da aula iremos acessar e processar um arquivo NetCDF do sensor ABI do satélite GOES-16/19 e visualizar a imagem na projeção original (conhecida como projeção "GOES-R", "satélite" ou "Geoestacionária"). Estes dados serão baixados do FTP da Amazon [GOES-16](https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-CMIPF/) e [GOES-19](https://noaa-goes19.s3.amazonaws.com/index.html#ABI-L2-CMIPF/).

- No FTP da Amazon os arquivos estão separados na seguinte forma:
   > `ano`: 2017 à 2025
   >> `dia juliano`: 0 à 366
   >>> `hora do dia`: 0 à 23

- Por exemplo: para o ano de `2019`, `dia juliano 110` e `22 horas`, temos o seguinte caminho do diretório:
`https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-CMIPF/2019/110/22/`

- Ao acessar este diretório observamos que um dos arquivos disponível é o `OR_ABI-L2-CMIPF-M6C13_G16_s20191102240230_e20191102249549_c20191102250031.nc`. A seguir vamos entender cada item do nome do arquivo.

- `OR_ABI-L2-CMIPF-M6C13_G16`:
    1. `OR`: Significada On-Readout (ou, em algumas interpretações, "Observation Record"). Indica que os dados foram processados a partir da transmissão direta do satélite, em tempo real, para as estações terrestres de recepção.

    2. `ABI`: Significa Advanced Baseline Imager. É o nome do instrumento a bordo do satélite que coletou os dados. O ABI é a câmera principal do GOES-16/19, responsável por imagear a Terra em diferentes bandas espectrais.

    3. `L2`: Significa Level 2. Indica o nível de processamento dos dados. Dados de Nível 2 são derivados dos dados de Nível 1b (que são dados de radiância calibrados e geo-localizados) e representam produtos derivados, como temperatura da superfície do mar, conteúdo de gelo, neblina, etc. Neste caso, é um produto de *Cloud and Moisture Imagery*.

    4. `CMIPF`: Significa *Cloud and Moisture Image Product, Full Disk*. Este é o nome específico do produto. Trata-se do produto de imagem de nuvens e umidade. O "F" especifica que a imagem cobre o Disco Completo (Full Disk) da Terra, que é a área visível pelo satélite.

    5. `M6C13`: Significa Mode 6, Channel 13.
        - M6: Indica o Modo de Operação 6 do instrumento ABI. O GOES-16/19 opera em diferentes modos que definem com que frequência ele varre diferentes áreas. O Modo 6 é um modo de operação específico que define a cadência de imageamento para o disco completo, setores meso e de varredura flexível.
        - C13: Indica o Canal 13 do instrumento ABI. O Canal 13 é a banda do infravermelho (10.30 µm). É ideal para observar características do topo de nuvens profundas.

    6. `G16`: Significa GOES-16. Identifica o satélite de origem dos dados. O GOES-16 é também conhecido como GOES-East

- `s20191102240230_e20191102249549_c20191102250031.nc`:

    1. s20191102240230 - Tempo de Início (Start Time)
        - 2019: Ano (2019)
        - 110: Dia do ano (110 = 20 de Abril)
        - 2240230: Hora, Minuto, Segundo (22:40:23.0 UTC)
        - Traduzindo: A varredura da imagem começou em 20 de Abril de 2019, às 22:40:23.0 UTC.

    2.  e20191102249549 - Tempo de Término (End Time)
        - 2019: Ano (2019)
        - 110: Dia do ano (110 = 20 de Abril)
        - 2249549: Hora, Minuto, Segundo (22:49:54.9 UTC)
        - Traduzindo: A varredura da imagem terminou em 20 de Abril de 2019, às 22:49:54.9 UTC.

    3. c20191102250031 - Tempo de Criação (Creation Time)
        - 2019: Ano (2019)
        - 110: Dia do ano (110 = 20 de Abril)
        - 2250031: Hora, Minuto, Segundo (22:50:03.1 UTC)
        - Traduzindo: O arquivo de dados final foi gerado e fechado no sistema em 20 de Abril de 2019, às 22:50:03.1 UTC.


---



Agora que já sabemos um pouco como são os arquivos do satélite GOES, iremos gerar a nossa primeira imagem de satélite.

No código a seguir temos basicamente 5 "blocos":
1.   **Importação das bibliotecas necessárias**
2.   **Criação de diretórios de entrada e saída de dados**
3.   **Download de dados do sensor ABI do GOES**
4.   **Leitura do aquivo NetCDF do sensor ABI**
5.   **Plotagem da imagem**


---





O que podemos modificar facilmente?

**1. Data e hora do arquivo a ser baixado**

**2. Qual canal ABI desejamos baixar**

**3. Escala de cores ([colormaps](https://matplotlib.org/stable/users/explain/colors/colormaps.html) padrão da matplotlib), título, entre outras decorações da imagem**


---



In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs
from datetime import datetime
from utilities_goes16e19 import download_CMI
import os
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_1"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento: 2019-04-20 às 22:40 UTC
yyyymmddhhmn = '201904202240'

# define o satélite: GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                               LEITURA DO ARQUIVO
#========================================================================================================================#
data = xr.open_dataset(path)

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# usa a projeção geoestacionária do cartopy
# para o GOES-16: longitude central: -75.0 / altura do satellite: 35786023.0
# a extensão do Full Disk: (metade dos pixels full disk) X (tamanho do pixel em radianos) X (altura do satélite em metros) => 2712 * 0.000056 * 35786023.0 = 5434894.67527
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
img_extent = (-5434894.67527, 5434894.67527, -5434894.67527, 5434894.67527)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)

# escala de cores
colormap = "gray_r" # escala de cores na ordem reversa - branco para preto para o canal do IR

# plota imagem
img = ax.imshow(data['CMI'] - 273.15, origin='upper', vmin=-80, vmax=40, extent=img_extent, cmap=colormap)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='vertical', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(data.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda {band} (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/G{goes_number}_ch{band}_full_cinza_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

In [ ]:
# mostrando o dado do satélite
data

In [ ]:
# para acessar alguma informação, exemplo os dados de temperatura
data['CMI']

In [ ]:
# para acessar alguma informação, exemplo a altitude do satélite "nominal_satellite_height"
data['nominal_satellite_height'].values

---
> `EXERCÍCIO:`
1.   Gerar imagem para uma data diferente
2.   Testar outras paletas de cores

---

#**PARTE 2)**: Projeção Satélite em T-Realçada

Agora iremos plotar a imagem do GOES na projeção satélite com as cores coloridas, também conhecida como imagem T-Realçada. Este tipo de imagem tem a vantagem de facilitar a identificação da temperatura do topo das nuvens.

In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
from datetime import datetime
from utilities_goes16e19 import download_CMI, remap, loadCPT
import os
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                          CRIA DIRETÓRIO DE ENTRADA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_2"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento: 2024-04-30 às 18:00 UTC
yyyymmddhhmn = '201904202240'

# define o satélite GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                               LEITURA DO ARQUIVO
#========================================================================================================================#
data = xr.open_dataset(path)

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# usa a projeção geoestacionária do cartopy
# para o GOES-16: longitude central: -75.0 / altura do satellite: 35786023.0
# a extensão do Full Disk: (metade dos pixels full disk) X (tamanho do pixel em radianos) X (altura do satélite em metros) => 2712 * 0.000056 * 35786023.0 = 5434894.67527
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
img_extent = (-5434894.67527, 5434894.67527, -5434894.67527, 5434894.67527)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)

# converte o arquivo CPT para ser usado em Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# plota imagem
img = ax.imshow(data['CMI'] - 273.15, origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=colormap)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='vertical', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(data.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda {band} (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/G{goes_number}_ch{band}_full_Trealcada_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

---
> `EXERCÍCIO:`
1.   Gerar imagem para uma data diferente
2.   Testar outras paletas de cores

---

#**PARTE 3)**: Projeção Retangular em Níveis de Cinza


Nesta terceira parte iremos reprojetar a imagem ABI, da projeção satélite (ou GOES-R) para a projeção retangular do tipo "cilíndrica equidistante".

Para isto utilizaremos a função `remap` que esta no código `utilities_goes16e19.py` que baixamos do Github. A função `remap` utiliza a biblioteca [GDAL](https://gdal.org/en/stable/) para realizar esta tranformação de projeção. O dado reprojetado será visualizado em escala de cinza invertida (branco para temperaturas de brilho mais baixas e preto para temperaturas de brilho mais altas).

Para identificar melhor as características das nuvens de tempestades, iremos realizar um zoom numa determinada área da nossa imagem, centrada no estado da Paraíba. Este zoom é indicado pelo parâmetro `extent = [-40.00, -10.00, -30.00, -2.00]`, definido por *[min lon, min lat, max lon, max lat]* .



In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import datetime
from utilities_goes16e19 import download_CMI, remap
import numpy as np
import os

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_3"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento
yyyymmddhhmn = '201904202240'

# define o satélite: GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                         REPROJETA E FAZ A LEITURA DO ARQUIVO
#========================================================================================================================#
# área desejada (min lon, min lat, max lon, max lat)
extent = [-40.00, -10.00, -30.00, -2.00]

# chama a função que faz a reprojeção (file, variable, extent, resolution)
grid = remap(path, 'CMI', extent, 2)

# leitura do dado e transforma para °C
data = grid.ReadAsArray() - 273.15

#========================================================================================================================#
#                                                 PLOT THE IMAGE
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# projeção geoestacionária do cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# define a extensão da imagem
img_extent = [extent[0], extent[2], extent[1], extent[3]] # Min lon, Max lon, Min lat, Max lat

# escala de cores
colormap = "gray_r" # escala de cores na ordem reversa - branco para preto para o canal do IR

# plota imagem
img = ax.imshow(data, origin='upper', vmin=-80, vmax=40, extent=img_extent, cmap=colormap)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# plota estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda {band} (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/G{goes_number}_ch{band}_retangular_cinza_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

In [ ]:
# dados reprojetados
grid

In [ ]:
# mostra a matriz de dados de temperatura reprojetados
data

In [ ]:
# leitura do arquivo reprojetado
ds = xr.open_dataset(f'{input}/{file_name}_ret.nc', mask_and_scale=True).sel(lon=slice(extent[0], extent[2]), lat=slice(extent[3], extent[1]))
ds

In [ ]:
# plot simples do mapa em projeção retangular
ds['Band1'].plot(cmap='jet')

---
> `EXERCÍCIO:`
1.   Alterar a região desejada (variável "extent")
---

#**PARTE 4)**: Projeção Retangular em T-Realçada

Nesta quarta parte vamos reprojetar a imagem e visualizá-la em escala colorida (T-realçada), realçando a imagem IR.

Para isso, vamos utilizar a função `loadCPT` que esta no código `utilities_goes16e19.py` que baixamos do Github para carregar uma paleta de cores no formato de arquivo .cpt`. Podemos acessar dezenas de escalas neste formato na seguinte página: http://seaviewsensing.com/pub/cpt-city/

In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import datetime
from utilities_goes16e19 import download_CMI, remap, loadCPT
import numpy as np
import os

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_4"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento
yyyymmddhhmn = '201904202240'

# define o satélite: GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                         REPROJETA E FAZ A LEITURA DO ARQUIVO
#========================================================================================================================#
# área desejada (min lon, min lat, max lon, max lat)
extent = [-40.00, -10.00, -30.00, -2.00]

# chama a função que faz a reprojeção (file, variable, extent, resolution)
grid = remap(path, 'CMI', extent, 2)

# leitura do dado e transforma para °C
data = grid.ReadAsArray() - 273.15

#========================================================================================================================#
#                                                 PLOT THE IMAGE
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# projeção geoestacionária do cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# define a extensão da imagem
img_extent = [extent[0], extent[2], extent[1], extent[3]] # Min lon, Max lon, Min lat, Max lat

# converte o arquivo CPT para ser usado em Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# plota imagem
img = ax.imshow(data, origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=colormap)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# plota estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda {band} (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/G{goes_number}_ch{band}_retangular_trealcada_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

---
> `EXERCÍCIO:`
1.   Alterar a região desejada (variável "extent")
---

#**PARTE 5)**: Plotando várias imagens do IR e criando animação

No exemplo anterior plotamos apenas uma única imagem. Porém, se quisermos produzir 5, 10 ou 20 imagens, como poderíamos fazer? Teríamos que mudar o horário e produzir uma de cada vez?

Para isto usaremos o conceito de loop e processaremos as imagens de `2019-04-20 22:40 UTC` à `2019-04-21 00:20 UTC` a cada **10min**. O loop com as datas das imagens será gerado usando a função `date_range` da famosa biblioteca `pandas`.

Colocaremos um contador de tempo no início da célula de código para verificar quanto tempo gastará para o processamento, `%%time`.

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import timedelta, datetime
from utilities_goes16e19 import download_CMI, download_GLM, remap, loadCPT
import numpy as np
import os

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_5"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE OS LIMITES DA IMAGEM
#========================================================================================================================#
# canal
band = '13'

# área desejada da imagem
lonmin, lonmax, latmin, latmax = -40.00, -30.00, -10.00, -2.00

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

#========================================================================================================================#
#                                              LOOP DAS IMAGENS
#========================================================================================================================#
# Loop das imagens
for date_image in pd.date_range('201904202240', '201904210030', freq='10min'):

    #--------------------------------------------------------------------------#
    #                          DATA E HORÁRIO
    #--------------------------------------------------------------------------#
    # data
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # '202404301300'

    # define o satélite: GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # extrai o ano, mês, dia, hor e min
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

    print('#=====================================================================================================#')
    print(f'                           PROCESSANDO A IMAGEM = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # download do arquivo
    file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

    # caminho do arquivo que foi baixado
    path = f'{input}/{file_name}.nc'

    #--------------------------------------------------------------------------#
    #                    REPROJETA OS DADOS DO ABI
    #--------------------------------------------------------------------------#
    # chama a função que faz a reprojeção (file, variable, extent, resolution)
    grid = remap(path, 'CMI', extent, 2)

    # leitura do dado e transforma para °C
    data = grid.ReadAsArray() - 273.15

    #--------------------------------------------------------------------------#
    #                           PLOTA A IMAGEM
    #--------------------------------------------------------------------------#
    # tamanho da figura (largura x altura em polegadas)
    plt.figure(figsize=(14,11))

    # projeção geoestacionária do cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    # converte o arquivo CPT para ser usado em Python
    cpt = loadCPT('ir.cpt')
    cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

    # plota imagem
    img = ax.imshow(data, origin='upper', vmin=-103.0, vmax=84, extent=[lonmin, lonmax, latmin, latmax], cmap=cmap, alpha=1.0)

    # legenda
    ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

    # linhas costeiras, bordas e linhas de grade do mapa
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # plota estados
    shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

    # barra de cores
    plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.055)

    # leitura da data/horário do arquivo NetCDF como uma string
    date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

    # título da figura
    plt.title(f'GOES-{goes_number} Banda {band} (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
    plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

    # salva figura
    plt.savefig(f'{output}/G{goes_number}_ch{band}_retangular_trealcada_{yyyy}-{mm}-{dd}_{hh}:{mn}_UTC.jpg', bbox_inches='tight', dpi=300)
    print('\n')

- O objetivo aqui é montar uma animação com as imagens de satélites que foram geradas. Para isto usaremos a biblioteca [imageio](https://imageio.readthedocs.io/en/stable/).

In [ ]:
# importa bibliotecas
import imageio
import glob

# lista as imagens que serão usadas na animação
files = sorted(glob.glob(f'{output}/*jpg'))

# faz animação
images = []
for file in files:
    images.append(imageio.imread(file))

# salva animação
imageio.mimsave(f'{output}/animacao.gif',
                images,
                duration=300,
                loop=0)

# mostra a animação
print("\nAbrindo o GIF..\n")
from IPython.display import Image
Image(open(f'{output}/animacao.gif','rb').read(), width=600)


---
> `EXERCÍCIO:`
1.   Alterar o intervalo de datas
2.   Alterar a duração dos frames da animação
---

#**PARTE 6)**: Plotando painel

É comum termos que mostrar várias imagens em apenas uma figura, em forma de painel. Nesse exemplo, aprenderemos como plotar 12 imagens em uma única figura. Mas antes iremos conhecer a bilioteca `Ultraplot` para plotar gráficos de alta qualidade.

![Texto alternativo](https://github.com/evmpython/Minicurso_UFCG_nov_2025/blob/main/logo/ultraplot.png?raw=true)

O [UltraPlot](https://github.com/Ultraplot/ultraplot) é uma biblioteca para produzir gráficos **bonitos** e de **alta qualidade** científica de **maneira fácil**. A grande vantagem do UltraPlot em relação ao Matplotlib para plotar gráficos é a sua sintaxe sucinta e a alta qualidade dos gráficos. Para maiores informações sobre o UltraPlot acesse:

- Pypi: https://pypi.org/project/ultraplot/

- GitHub: https://github.com/Ultraplot/

- Documentação: https://ultraplot.readthedocs.io/en/latest/

In [ ]:
# verificando se o ultraplot esta instalado no colab
!pip show ultraplot

In [ ]:
# instalando o ultraplot
!pip install -q ultraplot

In [ ]:
# verificando a versão do ultraplot disponível no colab
!pip show ultraplot

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
from utilities_goes16e19 import download_CMI, remap, loadCPT
from matplotlib import cm
import ultraplot as uplt
import pandas as pd
import os
from datetime import datetime
import xarray as xr
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_6"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE OS LIMITES DA IMAGEM
#========================================================================================================================#
# canal
band = '13'

# área desejada da imagem
lonmin, lonmax, latmin, latmax = -40.00, -30.00, -10.00, -2.00

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

# cria moldura da figura
fig, ax = uplt.subplots(ncols=3, nrows=4, axheight=6, tight=True, proj='pcarree')

# formatação dos eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=3, lonlines=3,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='40px', large='45px',
          suptitle='GOES CH13 (10.35 µm)')

# carrega tabela de cores
cpt_ir = loadCPT(f'/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

#========================================================================================================================#
#                                              LOOP DAS IMAGENS
#========================================================================================================================#
# Loop das imagens
for i, date_image in enumerate(pd.date_range('201904202240', '201904210030', freq='10min')):

    #--------------------------------------------------------------------------#
    #                          DATA E HORÁRIO
    #--------------------------------------------------------------------------#
    # data
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # 201904202240

    # define o satélite: GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # extrai o ano, mês, dia, hor e min
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

    print('#=====================================================================================================#')
    print(f'                           PROCESSANDO A IMAGEM = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # download do arquivo
    file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

    # caminho do arquivo que foi baixado
    path = f'{input}/{file_name}.nc'

    #--------------------------------------------------------------------------#
    #                    REPROJETA OS DADOS DO ABI
    #--------------------------------------------------------------------------#
    # chama a função que faz a reprojeção (file, variable, extent, resolution)
    grid = remap(path, 'CMI', extent, 2)

    # remove os arquivos desnecessários
    !rm {input}/{file_name}.nc
    !rm {input}/{file_name}.nc.aux.xml

    #--------------------------------------------------------------------------#
    #                         Leitura do dado
    #--------------------------------------------------------------------------#
    # leitura do arquivo reprojetado
    dataset_ir = xr.open_dataset(f'{input}/{file_name}_ret.nc', mask_and_scale=True).sel(lon=slice(lonmin, lonmax), lat=slice(latmax, latmin))

    # aplica o fator de escala (/10) e transforma para Graus Celsius
    dataset_ir['Band1'] = (dataset_ir['Band1']/10.) - 273.15

    #--------------------------------------------------------------------------#
    #                        Plotando a figura
    #--------------------------------------------------------------------------#
    if i == 0:
        map1 = ax[i].imshow(dataset_ir['Band1'],
                            cmap=cmap_ir,
                            extent=[lonmin, lonmax, latmin, latmax],
                            levels=uplt.arange(-103.0, 105, 1.0))
    else:
        ax[i].imshow(dataset_ir['Band1'],
                     cmap=cmap_ir,
                     extent=[lonmin, lonmax, latmin, latmax],
                     levels=uplt.arange(-103.0, 105, 1.0))

    # plota contornos dos Estados
    shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFCG_nov_2025/raw/main/utils/BR_UF_2019.shp').geometries())
    ax[i].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='yellow', facecolor='none', linewidth=2.1)

    # plota titulo de cada figura
    if (i==1 or i==2 or i==4 or i==5 or i==7 or i==8): ax[i].format(title=f'{yyyy}-{mm}-{dd} {hh}:{mn} UTC', labels=False, linewidth=3)
    if (i==10 or i==11): ax[i].format(title=f'{yyyy}-{mm}-{dd} {hh}:{mn} UTC', labels=[False, False, True, False], linewidth=3)
    if (i==0 or i==3 or i==6): ax[i].format(title=f'{yyyy}-{mm}-{dd} {hh}:{mn} UTC', labels=[True, False, False, False], linewidth=3)
    if (i==9): ax[i].format(title=f'{yyyy}-{mm}-{dd} {hh}:{mn} UTC', labels=[True, False, True, False], linewidth=3)
    print('\n')

# plota barra de cores
fig.colorbar(map1, loc='b', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=40, labelsize=40, width=0.5, length=0.7)

# salva figura
fig.savefig(f'{output}/painel_ch{band}.jpg', dpi=300, bbox_inches='tight')

---
> `EXERCÍCIO:`
1.   Alterar a área desejada
---

#**PARTE 7)**: Projeção Satélite da Imagem Realçada + Total de Flashes do GLM



---


Nesta parte da nossa aula iremos plotar a imagem do canal CH13 do sensor ABI do satélite GOES com as `ocorrências de flashes (relâmpagos)` registradas pelo sensor GLM do GOES. Os dados de relâmpagos são processados e disponibilizados pela NOAA no FTP da Amazon para o [GOES-16](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/) e [GOES-19](https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/).

Estes dados estão em arquivos netCDF a cada 20s, contendo as ocorrências de flashes, grupos e eventos nas Américas. Exemplo: `OR_GLM-L2-LCFA_G16_s20210101300000_e20210101300204_c20210101300214.nc`

- `OR_GLM-L2-LCFA_G16`:
    1.    `OR`: Significa On-Readout. São dados processados a partir da transmissão direta do satélite em tempo real.

    2.   `GLM`: Significa *Geostationary Lightning Mapper*. É o nome do instrumento que coletou os dados. O GLM é o primeiro instrumento de imageamento de relâmpagos em órbita geoestacionária, projetado para detectar continuamente a totalidade dos relâmpagos (nuvem-nuvem e nuvem-solo) sobre as Américas e oceanos adjacentes.

    3.   `L2`: Significa Level 2. Indica que são dados de nível 2, ou seja, um produto geofísico derivado. Os dados brutos (nível 1b) foram processados para gerar informações sobre a localização e energia dos eventos de relâmpagos.

    4.   `LCFA`: Significa *Lightning Cluster Filter Algorithm*. Este é o produto principal do GLM. O "LCFA" é o algoritmo que agrupa os pixels individuais de luz detectados (chamados "events") em aglomerados significativos, que representam: Events: A detecção bruta de um único pixel em um instante. Groups: Um aglomerado de "events" no mesmo flash que ocorrem quase simultaneamente. Flashes: A agregação completa de "groups" que constituem um único flash de relâmpago.

    4.   `G16`: Significa GOES-16. Satélite de origem dos dados (GOES-East).

- `s20210101300000_e20210101300204_c20210101300214.nc`: Semelhantemente aos dos do ABI indica a data do início, fim e gravação do scan.


Os dados de relâmpagos são baixados através da função `download_GLM` que esta no código `utilities_goes16e19.py` que baixamos do github do curso. Os relâmpagos são contabilizados num intervalo de 10min iniciando com a data da imagem. Por exemplo, na imagem de satélite `2019-04-20 22:40 UTC` os relâmpagos são acumulados das `22:40 UTC` às `22:50 UTC`.


---







![Texto alternativo](https://github.com/evmpython/Minicurso_UFCG_nov_2025/blob/main/logo/geobr_logo_y.png?raw=true)

Além dos dados de relâmpagos, iremos plotar o shapefile de algum município no nosso mapa. Paa isto utilizaremos a base de shapefiles disponibilizado pela biblioteca [geobr](https://pypi.org/project/geobr/). A geobr disponibiliza o shapefile de todos muncícipios, regiões metropolitanas, estados, meso e micro regiões do Brasil.



---



In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import timedelta, datetime
from utilities_goes16e19 import download_CMI, download_GLM, remap, loadCPT
import numpy as np
import os
import pandas as pd
import geobr

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_7"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento
yyyymmddhhmn = '201904202240'

# define o satélite: GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                         REPROJETA E FAZ A LEITURA DO ARQUIVO
#========================================================================================================================#
# área desejada da imagem
lonmin, lonmax, latmin, latmax = -40.00, -30.00, -10.00, -2.00

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

# chama a função que faz a reprojeção (file, variable, extent, resolution)
grid = remap(path, 'CMI', extent, 2)

# leitura do dado e transforma para °C
data = grid.ReadAsArray() - 273.15

#========================================================================================================================#
#                                              DOWNLOAD DOS DADOS DO GLM
#========================================================================================================================#
# inicializa os arrays de latitude e longitude dos flashes
lats_flash = np.array([])
lons_flash = np.array([])

# extrai o ano, mês, dia, hora, minuto e segundos
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')
date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
date_loop = date_ini

# Loop nos arquivos do GLM
while (date_loop <= date_end):

    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # Download o arquivo
    file_glm = download_GLM(yyyymmddhhmnss, goes_number, input)

    # Verifica se o arquivo existe antes de processar
    file_path = f'{input}/{file_glm}.nc'
    if os.path.exists(file_path):

        # leitura do arquivo
        glm_20s = xr.open_dataset(file_path)

        # appenda as lats / longs
        lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
        lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

        # fecha o arquivo
        glm_20s.close()
    else:
        print(f"Arquivo não encontrado: {file_path}")

    # incrementa a variável the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região de interesse
df_flash_filtered = df[ (df['lon'] > extent[0]) & (df['lon'] < extent[2]) & (df['lat'] > extent[1]) & (df['lat'] < extent[3])]

# transforma o dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
#----------------------------------------------------------------#
#               Define a configuração do gráfico
#----------------------------------------------------------------#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(14,11))

# projeção geoestacionária do cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# define a extensão da imagem
#            lonmin,     lonmax,    latmin,    latmax
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# converte o arquivo CPT para ser usado em Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

#----------------------------------------------------------------#
#                       Plota mapa
#----------------------------------------------------------------#
img = ax.imshow(data, origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=colormap)

#----------------------------------------------------------------#
#                       Plota GLM
#----------------------------------------------------------------#
glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='o', s=20, facecolor='white', edgecolor='black', linewidth=1, alpha=0.8, zorder=2, label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

#----------------------------------------------------------------#
#                      Plota shapefiles
#----------------------------------------------------------------#
# carrega o shapefiles de todos municípios do Brasil
shapefile_municipios = geobr.read_municipality(year=2020)

# filtra apenas o município
shapefile_municipio = shapefile_municipios[shapefile_municipios['name_muni'] == 'Serra Grande']

# plota o município
shapefile_municipio.plot(ax=ax, edgecolor='cyan', facecolor='none', linewidth=2.0, label='Serra Grande', zorder=3)

# carrega o shapefile dos estados brasileiros. Usa a função read_state do geobr para obter os polígonos dos estados
shapefile_estados = geobr.read_state(year=2020)

# filtra os estados dentro da extensão do mapa. Cria uma máscara para selecionar apenas os estados que intersectam com a área do mapa
shapefile_estados_filtrados = shapefile_estados.cx[extent[0]:extent[2], extent[1]:extent[3]]

# plota estados
shapefile_estados_filtrados.plot(ax=ax, edgecolor='white', facecolor='none', linewidth=1.0, zorder=3)

#----------------------------------------------------------------#
#           Plota demais formatações do gráfico
#----------------------------------------------------------------#
# define os limites do eixo para a extensão da imagem
ax.set_xlim(extent[0], extent[2])  # lonmin, lonmax
ax.set_ylim(extent[1], extent[3])  # latmin, latmax

# legenda
ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

#----------------------------------------------------------------#
#                      Plota shapefiles
#----------------------------------------------------------------#
# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.055)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda 13 (10.3 µm) + GLM Flashes\nABI: {date}', fontweight='bold', fontsize=10, loc='left')
plt.title(f'GLM: {str(date_ini)} - {str(date_end)}', fontsize="10", loc="right")

#----------------------------------------------------------------#
#                    Salva figura
#----------------------------------------------------------------#
plt.savefig(f'{output}/G{goes_number}_ch{band}_retangular_trealcada_flashes_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

---
> `EXERCÍCIO:`
1.   Alterar a data da imagem
---

#**PARTE 8)**: Evolução Temporal da Temperatura de Brilho do IR e Flashes GLM

Nesta parte avaliaremos a evolução temporal da temperatura de bilho e flashes para uma tempestade que atingiu um determinado município.


## **Mapa**


In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import timedelta, datetime
from utilities_goes16e19 import download_CMI, download_GLM, remap, loadCPT
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import salem
import geobr

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = "/content/output/parte_8"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE OS LIMITES DA IMAGEM
#========================================================================================================================#
# canal
band = '13'

# área desejada da imagem
lonmin, lonmax, latmin, latmax = -40.00, -30.00, -10.00, -2.00

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

#========================================================================================================================#
#                                              CARREGA SHAPEFILES
#========================================================================================================================#
# carrega o shapefiles de todos municípios do Brasil
shapefile_municipios = geobr.read_municipality(year=2020)

# filtra apenas o município de Campina Grande
shapefile_municipio = shapefile_municipios[shapefile_municipios['name_muni'] == 'Serra Grande']

# carrega o shapefile dos estados brasileiros. Usa a função read_state do geobr para obter os polígonos dos estados
shapefile_estados = geobr.read_state(year=2020)

# filtra os estados dentro da extensão do mapa. Cria uma máscara para selecionar apenas os estados que intersectam com a área do mapa
shapefile_estados_filtrados = shapefile_estados.cx[extent[0]:extent[2], extent[1]:extent[3]]

#========================================================================================================================#
#                                              LOOP DAS IMAGENS
#========================================================================================================================#
# variaveis da evolução temporal da temperatura e flashes
temp_min_municipio, temp_mean_municipio, flash_total_municipio, time_images = [], [], [], []

# Loop das imagens
for date_image in pd.date_range('201904202240', '201904210030', freq='10min'):

    #--------------------------------------------------------------------------#
    #                          DATA E HORÁRIO
    #--------------------------------------------------------------------------#
    # data
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # '202404301300'

    # define o satélite: GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # extrai o ano, mês, dia, hor e min
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')
    time_images.append(f'{hh}:{mn}')

    print('#=====================================================================================================#')
    print(f'                          PROCESSANDO A IMAGEM = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # download do arquivo
    file_name = download_CMI(yyyymmddhhmn, band, goes_number, input)

    # caminho do arquivo que foi baixado
    path = f'{input}/{file_name}.nc'

    #--------------------------------------------------------------------------#
    #                    REPROJETA OS DADOS DO ABI
    #--------------------------------------------------------------------------#
    # chama a função que faz a reprojeção (file, variable, extent, resolution)
    grid = remap(path, 'CMI', extent, 2)

    # leitura do dado e transforma para °C
    data = grid.ReadAsArray() - 273.15

    #--------------------------------------------------------------------------#
    #                       BAIXA OS DADOS DO GLM
    #--------------------------------------------------------------------------#
    # Data da imagem atual e da próxima
    date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
    date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
    date_loop = date_ini

    # loop de cumulação do GLM
    lats_flash, lons_flash = np.array([]), np.array([])
    while (date_loop <= date_end):

        # data
        yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

        # Download o arquivo
        file_glm = download_GLM(yyyymmddhhmnss, goes_number, input)

        # Verifica se o arquivo existe antes de processar
        file_path = f'{input}/{file_glm}.nc'
        if os.path.exists(file_path):

            # leitura do arquivo
            glm_20s = xr.open_dataset(file_path)

            # appenda as lats / longs
            lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
            lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

            # fecha o arquivo
            glm_20s.close()
        else:
            print(f"Arquivo não encontrado: {file_path}")

        # incrementa a variável the date_loop
        date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    #--------------------------------------------------------------------------#

    # coloca os flashes num dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # seleciona os flahes da região de interesee
    df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

    # transforma o dataframe para array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    #--------------------------------------------------------------------------#
    #                           PLOTA A IMAGEM
    #--------------------------------------------------------------------------#
    # tamanho da figura (largura x altura em polegadas)
    plt.figure(figsize=(14,11))

    # projeção geoestacionária do cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    # converte o arquivo CPT para ser usado em Python
    cpt = loadCPT('ir.cpt')
    cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

    # plota imagem
    img = ax.imshow(data, origin='upper', vmin=-103.0, vmax=84, extent=[lonmin, lonmax, latmin, latmax], cmap=cmap, alpha=1.0)

    # plota glm
    glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='o', s=20, facecolor='white', edgecolor='black', linewidth=1, alpha=0.8, zorder=2, label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

    # legenda
    ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

    # linhas costeiras, bordas e linhas de grade do mapa
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # plota o município
    shapefile_municipio.plot(ax=ax, edgecolor='cyan', facecolor='none', linewidth=2.0, label='Santa Maria', zorder=3)

    # plota estados
    shapefile_estados_filtrados.plot(ax=ax, edgecolor='white', facecolor='none', linewidth=1.0, zorder=3)

    # define os limites do eixo para a extensão da imagem
    ax.set_xlim(extent[0], extent[2])  # lonmin, lonmax
    ax.set_ylim(extent[1], extent[3])  # latmin, latmax

    # barra de cores
    plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.055)

    # leitura da data/horário do arquivo NetCDF como uma string
    date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

    # título da figura
    plt.title(f'GOES-{goes_number} Banda 13 (10.3 µm) + GLM Flashes\nABI: {date}', fontweight='bold', fontsize=10, loc='left')
    plt.title(f'GLM: {str(date_ini)} - {str(date_end)}', fontsize="10", loc="right")

    # salva figura
    plt.savefig(f'{output}/G{goes_number}_ch{band}_retangular_trealcada_flashes_{yyyy}-{mm}-{dd}_{hh}:{mn}_UTC.jpg', bbox_inches='tight', dpi=300)

    # mostra figura na tela (descomente aqui para mostrar cada imagem)
    #plt.show()
    #--------------------------------------------------------------------------#
    #        EXTRAI A TEMPERATURA DENTRO DA CIDADE DE CAMPINA GRANDE
    #--------------------------------------------------------------------------#
    # leitura do arquivo reprojetado
    dataset_ir = xr.open_dataset(f'{input}/{file_name}_ret.nc', mask_and_scale=True).sel(lon=slice(lonmin, lonmax), lat=slice(latmax, latmin))

    # aplica o fator de escala (/10) e transforma para Graus Celsius
    dataset_ir['Band1'] = (dataset_ir['Band1']/10.) - 273.15

    # extrai os valores apenas dentro do município
    data_ir_municipio = dataset_ir['Band1'].salem.roi(shape=shapefile_municipio)

    # calcula a temperatura mínima e média
    temp_min, temp_mean = float(data_ir_municipio.min(('lon', 'lat'))), float(data_ir_municipio.mean(('lon', 'lat')))

    # appenda as variáveis
    temp_min_municipio.append(temp_min)
    temp_mean_municipio.append(temp_mean)

    #--------------------------------------------------------------------------#
    #         EXTRAI OS FLASHES QUE ESTÃO DENTRO DO MUNICÍPIO
    #--------------------------------------------------------------------------#
    # novo geodataframe
    df_flash_filtered_gpd = gpd.GeoDataFrame(df_flash_filtered.reset_index(),
                                             geometry=gpd.points_from_xy(df_flash_filtered.lon,
                                                                         df_flash_filtered.lat))

    # desigina o systema de coordenadas (CRS)
    df_flash_filtered_gpd.crs = shapefile_municipio.crs

    # aplica a máscara
    df_flash_filtered_gpd_municipio = gpd.overlay(df_flash_filtered_gpd, shapefile_municipio, how='intersection')

    # appenda os flashes
    flash_total_municipio.append(df_flash_filtered_gpd_municipio.shape[0])
    print('\n')

#--------------------------------------------------------------------------#
#                      INSERE NUM DATAFRAME
#--------------------------------------------------------------------------#
data_ts = {'time': time_images,
           'temp_min_municipio': temp_min_municipio,
           'temp_mean_municipio': temp_mean_municipio,
           'flash_total_municipio': flash_total_municipio}

df_ts = pd.DataFrame(data_ts)

In [ ]:
# flashes
df_flash_filtered

In [ ]:
# mostrando o dataframe com temperaturas e flashes
df_ts

## **Animação das imagens**

- O objetivo aqui é montar uma animação com as imagens de satélites que foram geradas na pasta **output**. Para isto usaremos a biblioteca [imageio](https://imageio.readthedocs.io/en/stable/).

In [ ]:
# importa bibliotecas
import imageio
import glob

# lista as imagens que serão usadas na animação
files = sorted(glob.glob(f'{output}/*jpg'))

# faz animação
images = []
for file in files:
    images.append(imageio.imread(file))

# salva animação
imageio.mimsave(f'{output}/animacao.gif',
                images,
                duration=300,
                loop=0)

# mostra a animação
print("\nAbrindo o GIF..\n")
from IPython.display import Image
Image(open(f'{output}/animacao.gif','rb').read(), width=600)

##**Evolução Temporal**

In [ ]:
# mostrando os dados que iremos utilizar
df_ts

In [ ]:
#==================================================================================#
#                        CONFIGURAÇÃO INICIAL
#==================================================================================#
# importa biblioteca
import matplotlib.pyplot as plt

# tamanho da figura
fig, ax1 = plt.subplots(figsize=(10, 6))

#==================================================================================#
#             PLOT-1: temperatura no eixo y primério (ax1)
#==================================================================================#
# plota figura
ax1.plot(df_ts['time'].values, df_ts['temp_min_municipio'], marker='o', color='blue', linestyle='--', label='Temp. Mínima')
ax1.plot(df_ts['time'].values, df_ts['temp_mean_municipio'], marker='o', color='black', linestyle='--', label='Temp. Média')

# nomes dos eixos X e Y
ax1.set_xlabel('Tempo (UTC)', color='black', size=15)
ax1.set_ylabel('Temperatura de Brilho ($^o$C)', color='black', size=15)

# configuração dos eixos X e Y
ax1.tick_params(axis='x', labelcolor='black', labelsize=15, rotation=30)
ax1.tick_params(axis='y', labelcolor='black', labelsize=15)

# legenda
ax1.legend(loc='upper right', frameon=False)

#==================================================================================#
#             PLOT-2: flash no eixo y secundário (ax2)
#==================================================================================#
# adiciona eixo adicional
ax2 = ax1.twinx()

# plota figura
ax2.plot(df_ts['time'].values, df_ts['flash_total_municipio'].values, marker='o', color='green', label='Total Flash')

# nome eixo Y
ax2.set_ylabel('Total Flashes (fl/10min)', color='black', size=15)

# configuração do eixo Y
ax2.tick_params(axis='y', labelcolor='black', labelsize=15)

# adiciona legenda
ax2.legend(bbox_to_anchor=(0.97, 0.90), frameon=False)

# título da figura
plt.title(f'Evolução Temporal da Temperatura de Brilho (ABI) e Flashes (GLM)')

#==================================================================================#
#                           SALVA E MOSTRA A FIGURA
#==================================================================================#
# salva a figura
plt.savefig(f'{output}/evolucacao_temporal_temperatura_e_flashes.png', bbox_inches='tight', dpi=300)

# mostra a figura
plt.show()